In [1]:
# Libraries
import re
import os
import nltk
import requests
import unidecode
import unicodedata
import contractions
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from collections import Counter
from nltk.corpus import stopwords
from googletrans import Translator
#from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings('ignore')

In [2]:
# loadinng credenttials as environmen variables
load_dotenv('twitter_kafka_credentials.env', override = True)

True

In [3]:
# to get city coordinates
#geolocator = Nominatim(user_agent = 'bmartin')

# Get current date
today = dt.date.today()
today = today.strftime("%Y-%m-%d")
today

'2021-11-28'

## 1. Twitter API

### 1.1 About the Twitter API

The Twitter API can be used to retrieve and analyze data, as well as engage with the conversation on Twitter. It provides access to a variety of different resources including:
- Tweets
- Users
- Direct Messages
- Lists
- Trends
- Media
- Places

The Twitter API currently consists of two supported versions, as well as different access tiers. 
- **Standard v1.1**: The legacy standard endpoints provide access to the following resources with the standard v1.1 offerings.
    
    - Get Tweet timelines
    - Curate a collection of Tweets
    - Filter realtime Tweets
    - Sample realtime Tweets
    - Manage and pull public account information
    - Get trends near a location
    - Get locations with trending topics
    - Get information about a place    
    
    
- **Twitter API v2 Early Access**: A new Twitter API is being build with a modern and more sustainable foundation as well as an improved developer experience. The first endpoints are now available within Early Access, and enable users to listen to and analyze the public conversation. Additional endpoints, features, and access levels will be released soon.
    
    - Ability to request specific objects and fields.
    - New and more detailed data objects
    - Advanced metrics return in Tweets (including impressions, video views, user profile and URL clicks)
    - Insights on Tweet topics with annotations (filter by topic using `entity` and `context` operators)
    - Improved conversation tracking
    - Academic Research product track (grants free access to full-archive search)
    - High confidence spam filtering

In [4]:
# getting twitter credentials
twitter_key = os.environ.get('api_key')
twitter_secret_key = os.environ.get('secret_key')
bearer_token = os.environ.get('bearer_token')

In [5]:
# getting twitter credentials
twitter_key1 = os.environ.get('api_key1')
twitter_secret_key1 = os.environ.get('secret_key1')
bearer_token1 = os.environ.get('bearer_token1')

## 1.2 Retrieve data
### 1.2.1 Search Tweets

The [search endpoint](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent) returns Tweets from the last seven days that match a search query. Parameters are listed below:
- `query`(required): rule for matching Tweets.
- `expansions`: Expansions enable users to request additional data objects that relate to the originally returned Tweets.
- `max_results`: The maximum number of results to be returned.
- `next_token`: This parameter is used to get the next 'page' of results. 
- `place.fields`: Enables to select specific place fields that will be delivered in each returned Tweet. 
- `tweet.fields`: Enables to select specific Tweet fields that will be delivered in each returned Tweet object.
- `user.fields`: Enables to select specific user fields that will be delivered in each returned Tweet.

In [6]:
def search_tweets(query, bearer_token = bearer_token, next_token = None):    
    
    """
    Function to request tweets according to a specific query.
    
    Inputs:
        - query: A string that will be used to find tweets.
                 Tweets must match this string to be returned.
        - bearer_token: Security token from Twitter API.
        - next_token: ID of the next page that matches the specified query.
        
    Outputs: Dictionary (json type) with the requested data.  
    """
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    # end point
    url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&"

    params = {
        # select specific Tweet fields from each returned Tweet object
        'tweet.fields': 'text,created_at,lang,possibly_sensitive', # public_metrics
        
        # maximum number of search results to be returned (10 - 100)
        'max_results': 100,
        
        # additional data that relate to the originally returned Tweets
        'expansions': 'author_id,referenced_tweets.id,geo.place_id',
        
        # select specific place fields 
        "place.fields": 'country,full_name,name',
        
        # select specific user fields
        "user.fields": 'location',
        
        # get the next page of results.
        "next_token": next_token
    }
    
    # request
    response = requests.get(url = url, params = params, headers = headers)

    # verify successfull request
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    else:
        return response.json()

In [7]:
# search term
search_tweet = search_tweets(query = "Black Widow")

# 4 main keys
search_tweet.keys()

dict_keys(['data', 'includes', 'meta'])

## 2. Generate some data

After having requested tweets data, some dataframes were generated with different data.

- **tweets**: Pandas dataframe with information about tweets. The data from this dataframe is the required to perform text classification as well as to generate some visualizations. Columns:
    - `text`: Tweet content.
    - `lang`: Tweets' original language. Although some tweets are not in English, during the pre-processing process tweets are translated from the origin language to English.
    - `possibly_sensitive`: Boolean. Specifies if the tweet might be sensitive. 
    - `tweet_id`: Tweet's unique identifier.
    - `created_at`: When the tweet was created (tweeted).
    - `type`: Type of tweet: original tweet, replied (reply from another tweet), quoted or retweeted.
    
    
- **users**: Pandas dataframe with users information. Columns:
    - `user_id`: User's unique identifier.
    - `username`: User's username.
    - `name`: Name that is displayed on Twitter.
    - `location`: User's location. In Twitter there is this field in the user's biography were users can specify their location. Maybe this location is where they were born, where they currently live or just a random place. However, many users do not really include a geographical location in there, some of them just write something else such as their pronouns. So this field do not necessary specifies a geographical location.


- **places**: Pandas dataframe about places where a tweet was tweeted.
    - `country`: Country where a tweet was tweeted.
    - `full_name`: City, countrty where a tweet was tweeted.
    - `geo.place_id`: Unique identifier of location.
    - `name`: City name where a tweet was tweeted.

In [8]:
def create_dataframes(json_tweets, today):
    
    """
    Function to create and organize different data into specific data frames.
    
    Inputs:
        - json_tweets: A dictionary with tweets data.
    
    Outputs: 
        - tweets: Pandas dataframe with relevant information about tweets (to
                  further perform text classification).
                  
        - users: Pandas dataframe with users information.
        
        - places (optional): Pandas dataframe about places where users tweeted. If not a 
                  single tweets contains the place where it was tweeted, then
                  this dataframe will not be returned.
    """

    # Not all users enable their location when tweeting, so
    # we need to check if there are available locations for
    # the tweets returned.
    if "places" in json_tweets['includes'].keys():
        
        # If the field exists, create a dataframe with the corresponding data
        places = pd.json_normalize(json_tweets['includes']['places']).rename(columns = {"id":"geo.place_id"})
        
        # Create users dataframe
        users = pd.json_normalize(json_tweets['includes']['users']).rename(columns = {"id":"user_id"})
    
        # Create df with tweet's data
        tweets = pd.json_normalize(json_tweets['data']).rename(columns = {"id":"tweet_id"})
        
        # Get tweet's type
        tweets['type'] = tweets.referenced_tweets.apply(lambda x: x[0]["type"] if type(x) == list else None)
        
        # Drop retweeted tweets
        tweets = tweets[tweets["type"] != "retweeted"].reset_index(drop = True)
        
        # List of users in tweets dataframe to only 
        # keep users from tweets dataframe
        user_list = tweets.author_id.unique()
        users = users.loc[users.user_id.isin(user_list)].reset_index(drop = True)
        
        # Drop cols
        tweets = tweets.drop(['referenced_tweets','author_id','geo.place_id'], axis = 1)
        return tweets, users, places
    
    # Only return users and tweets dataframes since any tweet 
    # contained information about the place where it was tweeted.
    else: 
        # Create users dataframe
        users = pd.json_normalize(json_tweets['includes']['users']).rename(columns = {"id":"user_id"})
    
        # Create df with tweet's data
        tweets = pd.json_normalize(json_tweets['data']).rename(columns = {"id":"tweet_id"})
        
        # Get tweet's type
        tweets['type'] = tweets.referenced_tweets.apply(lambda x: x[0]["type"] if type(x) == list else None)
        
        # Drop retweeted tweets
        #tweets = tweets[tweets["type"] != "retweeted"].reset_index(drop = True)
        
        # List of users in tweets dataframe
        user_list = tweets.author_id.unique()

        # Only keep users from tweets dataframe
        users = users.loc[users.user_id.isin(user_list)].reset_index(drop = True)
        
        # Drop cols
        tweets = tweets.drop(['referenced_tweets','author_id'], axis = 1)
        return tweets, users

In [9]:
# Check if we have tweet's location
if "places" in search_tweet['includes'].keys():
    main_tweets, main_users, main_places = create_dataframes(search_tweet, today)
    
else:
    main_tweets, main_users = create_dataframes(search_tweet, today)
    main_places = pd.DataFrame()

In [10]:
print(main_tweets.shape)
print(main_users.shape)
#print(main_places.shape)

(100, 6)
(99, 4)


### 2.1 Request more data
The `search_tweets` function was build to only resquest tweets one time, nevertheless, with the `next_token` parameter we can easily request more data. This parameter indicates that there are more "pages" or more results (tweets) that matches the query it was previously sent to Twitter API. If the `next_token` parameter is found in the returned dictionary, then it means there are more results than the ones first returned. If this parameter is missing, then there are no more tweets regarding this topic.

In [11]:
# for i in range(1, 16) # ~750
# for i in range(1, 21): # ~1000
for i in range(1, 31):
    
    # Check if there is a next token (another page)
    # that matches the desired query
    if 'next_token' in search_tweet['meta'].keys():
        print(i, search_tweet["meta"]["next_token"])

        # Collect data from next token
        new_tweets = search_tweets(query = "Black Widow", next_token = search_tweet['meta']['next_token'])
        search_tweet = new_tweets

        # Check if any tweet has enabled the location,
        # so we can create the places dataframe.
        if "places" in search_tweet['includes'].keys():
            tweets, users, places = create_dataframes(search_tweet, today = today)

            # Append data to main tweets
            main_tweets = main_tweets.append(tweets)
            main_users = main_users.append(users)
            main_places = main_places.append(places)

            # Reset index
            main_tweets = main_tweets.reset_index(drop = True)
            main_users = main_users.reset_index(drop = True)
            main_places = main_places.reset_index(drop = True)

        # If any tweet has its location enabled, then only
        # create the other two dataframes.
        else: 
            tweets, users = create_dataframes(search_tweet, today = today)

            # Append data to main tweets
            main_tweets = main_tweets.append(tweets)
            main_users = main_users.append(users)

            # Reset index
            main_tweets = main_tweets.reset_index(drop = True)
            main_users = main_users.reset_index(drop = True)

    # If there are not more results regarding the
    # requested topic, then just stop requesting 
    # more data.
    else:
        break

1 b26v89c19zqg8o3fpdy7o05sa8x65gw7tv2ku33a088e5
2 b26v89c19zqg8o3fpdy7o056yh9o5asmxno5dx90pq7p9
3 b26v89c19zqg8o3fpdy7o04w4kbzn2voh1ipfxpkv8k1p
4 b26v89c19zqg8o3fpdy7o04l7l39cghor9pv2blplxmd9
5 b26v89c19zqg8o3fpdy7o04ac4wgby8p5xwzzuovosfwd
6 b26v89c19zqg8o3fpdy7o03zgn7q8ac6ob37au1s0gfi5
7 b26v89c19zqg8o3fpdy7ny1nywaqealk664d3c0lrhxfh
8 b26v89c19zqg8o3fpdy7ny1d4x8jhastbo69sxp3n9uv1
9 b26v89c19zqg8o3fpdy7ny12ch7uoce6fqu2hlvkhu8al
10 b26v89c19zqg8o3fpdy7ny0rikxhhlilv6kz24udczrel
11 b26v89c19zqg8o3fpdy7ny0glk6yer1vmn79bxv65gwal
12 b26v89c19zqg8o3fpdy7ny05t5o5mocb440yn84kplnct
13 b26v89c19zqg8o3fpdy7nxzv27uksjviyyevc39r7v5h9
14 b26v89c19zqg8o3fpdy7nxzk8ax74erjp2cwid0c3k871
15 b26v89c19zqg8o3fpdy7nxz9bb1c0e8rs398obyweb9fh
16 b26v89c19zqg8o3fpdy79bmju39w2k7mcag05tnozxw8t
17 b26v89c19zqg8o3fpdy79bm935vm5sqjey59kxu81txtp
18 b26v89c19zqg8o3fpdy79blydrr6njc017yrvc7epid8d
19 b26v89c19zqg8o3fpdy79blnjv8t1rq7u48aaaykgau0t
20 b26v89c19zqg8o3fpdy79blcmw7zxszxh3qasp3b74ol9
21 b26v89c19zqg8o3fpdy79bl1vx

In [12]:
print(main_tweets.shape)
print(main_users.shape)
print(main_places.shape)

(2350, 6)
(2246, 4)
(20, 4)


In [13]:
xxxxxxxxxxxxxxxxxxxxxxxx

NameError: name 'xxxxxxxxxxxxxxxxxxxxxxxx' is not defined

In [17]:
# Store the data locally
if main_places.empty:
    main_tweets.to_csv(f"data/tweets/tweets_{today}_part3.csv", index = False)
    main_users.to_csv(f"data/users/users_{today}_part3.csv", index = False)
    
else:
    main_tweets.to_csv(f"data/tweets/tweets_{today}_part3.csv", index = False)
    main_users.to_csv(f"data/users/users_{today}_part3.csv", index = False)
    main_places.to_csv(f"data/places/places_{today}_part3.csv", index = False)

### 2.3 Clean data
Before implementing the algorithm, we should start by cleaning and pre-processing our data, in this case, the papers csv is already loaded. The pre-processing phase includes the following steps and it's performed with help of the `PreProcessor` class:

- **Remove noise:** Noise removal is about removing characters digits and pieces of text that can interfere with text analysis. Noise removal is one of the most essential text preprocessing steps.


- **Normalize text:** Text normalization is the process of transforming a text into a canonical (standard) form. For example, the word “gooood” and “gud” can be transformed to “good”, its canonical form. 


- **Tokenization:** Tokenization is a way of separating a piece of text into smaller units called tokens. In this case tokens are words (but can also be characters or subwords).


- **Stemming:** Stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words (known as a lemma).


- **Lemmatization:** Lemmatization is a method responsible for grouping different inflected forms of words into the root form, having the same meaning. It is similar to stemming.

In [15]:
class PreProcessor:
    
    def __init__(self, regex_dict = None):
        
        # creating classes
        # stem
        self.sb = nltk.stem.SnowballStemmer('english')
        
        # lemmatize
        self.lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
        
        # translate
        self.translator = Translator()
        
        # declare a default regex dict
        self.default_regex_dict = {'goo[o]*d':'good', '2morrow':'tomorrow', 'b4':'before', 'otw':'on the way',
                                   'idk':"i don't know", ':)':'smile', 'bc':'because', '2nite':'tonight',
                                   'yeah':'yes', 'yeshhhhhhhh':'yes', ' yeeeee':'yes', 'btw':'by the way', 
                                   'fyi':'for your information', 'gr8':'great', 'asap':'as soon as possible', 
                                   'yummmmmy':'yummy', 'gf':'girlfriend', 'thx':'thanks','nowwwwwww':'now', 
                                   ' ppl ':' people ', 'yeiii':'yes'}
        
        # if no regex_dict defined by user, then use 
        # one by default. Else, concat two regex dicts
        if regex_dict:            
            self.regex_dict = {**regex_dict, **default_regex_dict}
            
        else:
            self.regex_dict = self.default_regex_dict
    
    def translate_twt(self, pdf):
    
        """
        This function helps to translate a tweet from any 
        language to English.

        Inputs:
            - pdf: Pandas dataframe. This dataframe must have
               the following columns:
                - lang: Tweet's language.
                - clean_tweet: Partially pre-processed tweet.

        Outputs: Translated tweet from any language available 
                 in googletrans api to English.
        """

        # Check if the language of the tweet is either undefined or English
        # to avoid translation.
        if pdf["lang"] == "und" or pdf["lang"] == "en":
            pdf["translated_tweet"] = pdf["clean_tweet"]

        # Check if tweet is in Hindi. The code of Hindi language is "hi", but 
        # Twitter has defined the code as "in".
        elif pdf["lang"] == "in":
            pdf["translated_tweet"] = self.translator.translate(pdf["clean_tweet"], src = "hi", dest = "en").text
            
        # Check if tweet is in Chinese. 
        # The api supports simplified and traditional chinese.
        elif pdf["lang"] == "zh":
            pdf["translated_tweet"] = self.translator.translate(pdf["clean_tweet"], src = "zh-cn", dest = "en").text

        # For any other language the translator should work just fine, so the
        # api should work with the language detected by Twitter.
        else:
            pdf["translated_tweet"] = self.translator.translate(pdf["clean_tweet"], src = pdf["lang"], dest = "en").text

        return pdf["translated_tweet"]

    
    def removeNoise(self, pdf):
        
        """
        Function to remove noise from strings. 
        
        Inputs: A pandas dataframe with raw strings of length n.
        
        Output: A clean string where elements such as accented 
        words, html tags, punctuation marks, and extra white 
        spaces will be removed (or transform) if it's the case.
        """
        
        # to lower case
        pdf["clean_tweet"] = pdf.text.apply(lambda x: x.lower())
        
        # remove accented characters from string
        # e.g. canción --> cancion
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: unidecode.unidecode(x))
        
        # remove html tags 
        pdf["clean_tweet"] = pdf.clean_tweet.str.replace(r'<[^<>]*>', '', regex=True)
        
        # remove (match with) usernames | hashtags | punct marks | links
        # punct marks = ",.':!?;
        # do not remove: ' 
        # but remove: "
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x:' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([-.,:_;])|(https?:\/\/.*[\r\n]*)",
                                                                            " ", x).split()).replace('"',''))
                
        # remove white spaces at the begining and at 
        # the end of a string
        pdf['clean_tweet'] = pdf.clean_tweet.apply(lambda x: x.lstrip(' '))
        pdf['clean_tweet'] = pdf.clean_tweet.apply(lambda x: x.rstrip(' '))
        
        # Translate tweet
        pdf["clean_tweet"] = pdf.apply(lambda x: self.translate_twt(x), axis = 1)
        
        # normalize string
        # normalize accented charcaters and other strange characters
        # NFKD if there are accented characters (????
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: unicodedata.normalize('NFKC', x).encode('ASCII', 'ignore').decode("utf-8"))
        
        return pdf
    
    
    def textNormalization(self, pdf):
        
        """
        Function to normalize a string. 
        
        Inputs: A pandas dataframe with strings (of length n) that 
        will be normalized. 
        
        Outputs: A normalized string whitout noise, words in their
        (expected) correct form and with no stopwords.
        """
        
        # remove noise first
        pdf = self.removeNoise(pdf)

        # expand contractions
        # e.g. don't --> do not
        pdf['clean_tweet'] = pdf.clean_tweet.apply(lambda x: contractions.fix(x))
 
        # Normalize words
        pdf['clean_tweet'] = pdf.clean_tweet.replace(self.regex_dict)
                
        # get English stopwords    
        stop_words = stopwords.words('english')
        stopwords_dict = Counter(stop_words)
        
        # remove stopwords from string
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: ' '.join([word for word in x.split()
                                                                       if word not in stopwords_dict]))
            
        return pdf
    
    
    def wordTokenize(self, pdf):
        """
        Function to tokenize a string into words. Tokenization is a way 
        of separating a piece of text into smaller units called tokens.
        In this case tokens are words (but can also be characters or 
        subwords).
        
        Inputs: A pandas dataframe with strings (of length n) that will be tokenized. 
        
        Outputs: A list of tokenized words.
        """
        # string normalized
        #normalized = self.textNormalization(string)
        pdf = self.textNormalization(pdf)
        
        # Use word_tokenize method to split the string
        # into individual words. By default it returns
        # a list.
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: nltk.word_tokenize(x))        
        
        # Using isalpha() will help us to only keep
        # items from the alphabet (no punctuation
        # marks). 
        #pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: [word for word in x if word.isalpha()])
        
        # Keep only unique elements
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: list(set(x)))

        # return list of tokenized words by row
        return pdf
    
    def phraseTokenize(self, pdf):
        
        """
        Function to tokenize a string into sentences. Tokenization is
        a way of separating a piece of text into smaller units called
        tokens. In this case tokens are phrases (but can also be words,
        characters or subwords).
        
        Inputs: A string (of length n) that will be tokenized. 
        
        Outputs: A list of tokenized sentences.
        """
        
        # pandas dataframe with strings normalized
        pdf = self.textNormalization(pdf)
        
        # Use sent_tokenize method to split the string
        # into sentences. By default it returns a list.
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: nltk.sent_tokenize(x))   
        
        return pdf 
    
    
    def stemWords(self, pdf):
        
        """
        Function to stem strings. Stemming is the process of reducing
        a word to its word stem that affixes to suffixes and prefixes 
        or to the roots of words (known as a lemma).
        
        Inputs: A raw string of length n.
        
        Output: Roots of each word of a given string.
        """
        
        # pandas dataframe with strings normalized
        pdf = self.textNormalization(pdf)
        
        # tokenized string (into words)
        pdf = self.wordTokenize(data)
            
        # reduct words to its root    
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: [self.sb.stem(word) for word in x])
        
        return pdf
    
    
    def lemmatizeWords(self, pdf):
        
        """
        Function to lemmatize strings. Lemmatization is a method 
        responsible for grouping different inflected forms of 
        words into the root form, having the same meaning. It is 
        similar to stemming.
        
        Inputs: A raw string of length n.
        
        Output: Roots of each word of a given string (with better
        performance than in stemming).
        """
        
        # pandas dataframe with strings normalized
        pdf = self.textNormalization(pdf)
        
        # list of tokenized words (from string)
        # Here it was decided to tokenize by words
        # rather than by sentences due to we thought
        # it would be easier to find the correct roots
        # of each word.
        pdf = self.wordTokenize(pdf)
        
        # lematize word from list of tokenized words
        #lematized = [self.lemmatizer.lemmatize(word) for word in tokenized]
        pdf["clean_tweet"] = pdf.clean_tweet.apply(lambda x: [self.lemmatizer.lemmatize(word) for word in x])
        
        return pdf

In [16]:
# Create class object
pre_processor = PreProcessor()

# Clean data and only keep 
# the roots of each word.
#tweets['clean_tweet'] = tweets.text.apply(pre_processor.removeNoise)
main_tweets = pre_processor.lemmatizeWords(main_tweets)
print(main_tweets.shape)
main_tweets.head(2)

(2350, 7)


,tweet_id,lang,possibly_sensitive,created_at,text,type,clean_tweet
0,1465177761631199238,tl,False,2021-11-29T04:36:14.000Z,grabe makahilak ang after credits sa black wid...,None,"[black, widow, makahilak, credit, grabe, ang, sa]"
1,1465177752881958919,th,False,2021-11-29T04:36:12.000Z,แพลนหลังสอบเสร็จ\n-Black Widow\n-ซีรีส์วันด้า\...,None,"[besrcch, widow, black, /, (, `, ncis, chiirii..."


In [18]:
# Store the data locally
main_tweets.to_csv(f'data/clean_tweets/clean_tweets_{today}_part3.csv', index = False)